In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, accuracy_score
from keras.preprocessing.image import ImageDataGenerator
import keras.utils as image
import PIL

In [2]:
#set up constants
targetSize = 28
color = 'rgb'
classMode = 'categorical'
trainingFiles = 'Vegetable_Images/train'

In [3]:
#preprocessiing

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True)

#trainingSet

training_set = train_datagen.flow_from_directory(
    trainingFiles,
    target_size = (targetSize, targetSize),
    batch_size = 32,
    class_mode = classMode,
    color_mode = color
)

#setup test
test_datagen = ImageDataGenerator(rescale=1./255)

test_set = test_datagen.flow_from_directory(
        'Vegetable_Images/test', 
        target_size=(targetSize, targetSize), 
        batch_size=32,
        class_mode=classMode,
        color_mode=color)

Found 15000 images belonging to 15 classes.
Found 3000 images belonging to 15 classes.


In [4]:
# create model

model = Sequential()

model.add(Conv2D(
    filters = 32,
    kernel_size = 3,
    activation = 'relu',
    input_shape = [targetSize, targetSize, 3]))

model.add(MaxPool2D(
    pool_size = 2,
    strides = 2
))

model.add(Conv2D(
    filters = 64,
    kernel_size = 3,
    activation = 'relu',
    input_shape = [targetSize, targetSize, 3]))

model.add(MaxPool2D(
    pool_size = 2,
    strides = 2
))

model.add(Conv2D(
    filters = 128,
    kernel_size = 3,
    activation = 'relu',
    input_shape = [targetSize, targetSize, 3]))

model.add(MaxPool2D(
    pool_size = 2,
    strides = 2
))

model.add(Flatten())

model.add(Dense(
    units = 128,
    activation = 'relu'))

model.add(Dense(
    units = 32,
    activation = 'relu'))

model.add(Dense(
    units = 15,
    activation = 'sigmoid'))

adam = Adam(learning_rate = 0.005)

Metal device set to: Apple M1


2023-03-13 13:32:14.877321: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-13 13:32:14.877574: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
#Compile model
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [13]:
#Train model
model.fit(x=training_set, epochs=30)

Epoch 1/30
469/469 [==============================] - 12s 26ms/step - loss: 0.3943 - accuracy: 0.8785
Epoch 2/30
469/469 [==============================] - 12s 26ms/step - loss: 0.3972 - accuracy: 0.8747
Epoch 3/30
469/469 [==============================] - 11s 24ms/step - loss: 0.3685 - accuracy: 0.8855
Epoch 4/30
469/469 [==============================] - 11s 24ms/step - loss: 0.3833 - accuracy: 0.8829
Epoch 5/30
469/469 [==============================] - 11s 24ms/step - loss: 0.3475 - accuracy: 0.8901
Epoch 6/30
469/469 [==============================] - 11s 24ms/step - loss: 0.3900 - accuracy: 0.8823
Epoch 7/30
469/469 [==============================] - 11s 23ms/step - loss: 0.3845 - accuracy: 0.8825
Epoch 8/30
469/469 [==============================] - 11s 24ms/step - loss: 0.3689 - accuracy: 0.8865
Epoch 9/30
469/469 [==============================] - 10s 21ms/step - loss: 0.3514 - accuracy: 0.8923
Epoch 10/30
469/469 [==============================] - 11s 22ms/step - loss: 0.360

In [14]:
#check accuracy

model.evaluate(test_set)

94/94 [==============================] - 2s 22ms/step - loss: 0.4468 - accuracy: 0.8830


[0.44675764441490173, 0.8830000162124634]

In [15]:
model.save("60epoch.h5")

In [26]:
rev_indicies = {v: k for k, v in training_set.class_indices.items()}


{0: 'Bean', 1: 'Bitter_Gourd', 2: 'Bottle_Gourd', 3: 'Brinjal', 4: 'Broccoli', 5: 'Cabbage', 6: 'Capsicum', 7: 'Carrot', 8: 'Cauliflower', 9: 'Cucumber', 10: 'Papaya', 11: 'Potato', 12: 'Pumpkin', 13: 'Radish', 14: 'Tomato'}


In [25]:
singlePred = 'Vegetable_Images/validation/Broccoli/1201.jpg'
test_image = image.load_img(singlePred, target_size = [targetSize, targetSize], color_mode = color)

test_image = image.img_to_array(test_image)

test_image = np.expand_dims(test_image, axis= 0)

results = model.predict(test_image/255.0)
index = np.argmax(results)

print(f'Vegetable: {rev_indicies[index]}')

1/1 [==============================] - 0s 12ms/step
Vegetable: Broccoli
